# Imports

In [208]:
import warnings

warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from scipy.linalg import svd
import sklearn.cluster as cluster
import plotly.express as px
import plotly.graph_objects as go

np.random.seed(1234)

In [209]:
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
# mpl.rcParams.update({"axes.grid": True})

# 8.1 Parabolic Fitting

## Data Load - Cars

In [210]:
def get_vehicle_data():
    vehicle_data = pd.read_csv("./data/auto-mpg.csv")
    vehicle_data["horsepower"] = pd.to_numeric(
        vehicle_data["horsepower"], errors="coerce"
    )
    vehicle_data.rename(columns={"horsepower": "hp"}, inplace=True)
    vehicle_data.dropna(inplace=True)
    return vehicle_data

In [211]:
vehicle_data = get_vehicle_data()
vehicle_data.head()

,mpg,cylinders,displacement,hp,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [212]:
vehicle_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 392 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   hp            392 non-null    float64
 4   weight        392 non-null    int64  
 5   acceleration  392 non-null    float64
 6   model year    392 non-null    int64  
 7   origin        392 non-null    int64  
 8   car name      392 non-null    object 
dtypes: float64(4), int64(4), object(1)
memory usage: 30.6+ KB


## Scatter Horsepower vs. MPG

In [213]:
px.scatter(
    vehicle_data,
    x="hp",
    y="mpg",
    trendline="ols",
    trendline_color_override="red",
)

# 8.2 Nonlinear Features

There is no non-linear scikit model, so we can generate our own non-linear features e.g. horsepower squared, then do linear fits including those columns

In [214]:
vehicle_data["hp2"] = vehicle_data["hp"] ** 2
vehicle_data.head()

,mpg,cylinders,displacement,hp,weight,acceleration,model year,origin,car name,hp2
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,16900.0
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,27225.0
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,22500.0
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,22500.0
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,19600.0


In [215]:
model = LinearRegression().fit(vehicle_data[["hp", "hp2"]], vehicle_data["mpg"])
[model.intercept_, model.coef_]

[56.90009970211294, array([-0.46618963,  0.00123054])]

In [216]:
[
    LinearRegression().fit(vehicle_data[["hp"]], vehicle_data["mpg"]).coef_,
    LinearRegression().fit(vehicle_data[["hp"]], vehicle_data[["mpg"]]).coef_,
]

[array([-0.15784473]), array([[-0.15784473]])]

In [217]:
from sklearn.metrics import mean_squared_error

mean_squared_error(vehicle_data["mpg"], model.predict(vehicle_data[["hp", "hp2"]]))

18.98476890761722

# 8.3 Prediction vs. Inference

2 distinct goals of modeling - prediction and inference
Prediction
- Model predicts accurate outcomes from real-world data
- For example, given a new previously unseen data point, model can make a reasonable prediction about how well it might perform

Inference
- Model helps understand the true relationship
- Answers key questions
- A much harder task

# 8.4 Scikit Transformers

In [218]:
from sklearn.preprocessing import PolynomialFeatures

vehicle_data = get_vehicle_data()
poly_transform = PolynomialFeatures(degree=3, include_bias=False)
df_cubic = pd.DataFrame(
    poly_transform.fit_transform(vehicle_data[["hp"]]),
    columns=poly_transform.get_feature_names_out(),
)
df_cubic.head()

,hp,hp^2,hp^3
0,130.0,16900.0,2197000.0
1,165.0,27225.0,4492125.0
2,150.0,22500.0,3375000.0
3,150.0,22500.0,3375000.0
4,140.0,19600.0,2744000.0


# 8.5 Scikit Pipelines

In [219]:
cu_model = LinearRegression(fit_intercept=True).fit(df_cubic, vehicle_data["mpg"])

## Predicting Values from New Data

The model thusly defined needs cubic features, but a new data point would just be the horsepower.  

You could build the features yourself using transformer, for example, or...

In [220]:
[
    cu_model.predict([[100, 100**2, 100**3]]),
    cu_model.predict(poly_transform.fit_transform([[100]])),
]

[array([22.44325879]), array([22.44325879])]

In [221]:
mean_squared_error(
    vehicle_data["mpg"],
    cu_model.predict(poly_transform.fit_transform(vehicle_data[["hp"]])),
)

18.94498981448592

## Scikit Pipeline

The above workarounds are a bit clunky  
Instead, use pipeline. Consider the above workflow a 2-stage pipeline:
- Generate features
- Do the linear regression fit

### Build the Pipeline

In [252]:
from sklearn.pipeline import Pipeline

pipelined_model = Pipeline(
    [
        ("transform", PolynomialFeatures(degree=3, include_bias=False)),
        ("regression", LinearRegression()),
    ]
)

pipelined_model.fit(vehicle_data[["hp"]], vehicle_data["mpg"])

Pipeline(steps=[('transform', PolynomialFeatures(degree=3, include_bias=False)),
                ('regression', LinearRegression())])

### Use to Make a New Prediction

In [223]:
pipelined_model.predict([[100]])

array([22.44325879])

### Predict All and Compute MSE

In [224]:
mpg_fit = pipelined_model.predict(vehicle_data[["hp"]])
mean_squared_error(vehicle_data["mpg"], mpg_fit)

18.94498981448592

### Pipeline Model Attributes

Attributes include
- Can be applied to new data directly
- Avoids need to explicitly create separate dataframe with transformer features
- Avoids need to keep track of various separate variables for transformer and regression objects

Downsides
- A bit more work to get at something for a part of the pipeline, for example the model coefs

In [225]:
pipelined_model.named_steps["regression"].coef_

array([-5.68850128e-01,  2.07901126e-03, -2.14662591e-06])

# 8.6 Order 0 - 6 Models on Vehicle Data

Variance in machine learning refers to sensitivity to the training data

Adding just a single outlier to a data set causes the higher-order model to change quite significantly, whereas the lower order models did not change much
- So the lower-order model has lower variance

Showing plot of model performance vs. complexity

![alt text](./data/performance_vs_complexity.png "Performance vs. Complexity")


# 8.7 Dangers of Overfitting

Given N data points, order N - 1 will give MSE = 0

![alt text](./data/fitting_as_linear_system.png "Fitting as Linear System")

Overfitting is basically memorization, just memorizing existing data, but cannot respond to new data at all


# 8.8 Detecting Overfitting with Simple Cross Validation

## To keep things simple and easy to visualize, choose just 35 data points from the car set

In [226]:
vehicle_data_35 = vehicle_data.sample(35, random_state=42)

## Define function to generate MSE given order

In [227]:
def get_mse_for_degree_k(
    k,
    df_fit: pd.DataFrame = vehicle_data_35,
    df_evaluate: pd.DataFrame = vehicle_data_35,
):
    pipelined_model = Pipeline(
        [
            ("transform", PolynomialFeatures(degree=k)),
            ("regression", LinearRegression(fit_intercept=True)),
        ]
    )

    pipelined_model.fit(df_fit[["hp"]], df_fit["mpg"])
    return mean_squared_error(
        df_evaluate["mpg"], pipelined_model.predict(df_evaluate[["hp"]])
    )

## Exercise for degrees 0,...,6

In [232]:
ks = list(range(7))
df_mses = pd.DataFrame({"k": ks, "MSE": [get_mse_for_degree_k(k) for k in ks]})
df_mses

,k,MSE
0,0,50.630727
1,1,19.542525
2,2,17.580688
3,3,17.114635
4,4,16.489650
5,5,15.438443
6,6,14.918056


In [229]:
fig = px.scatter(df_mses, x="k", y="MSE").update_traces(mode="lines+markers")
fig.show()

## Exercise model using new points

In [230]:
df_mses["MSE 9 Points"] = [
    get_mse_for_degree_k(k, df_evaluate=vehicle_data.sample(9, random_state=6))
    for k in ks
]
df_mses

,k,MSE,MSE 9 Points
0,0,50.630727,70.044988
1,1,19.542525,33.174596
2,2,17.580688,22.587845
3,3,17.114635,36.360989
4,4,16.489650,63.417542
5,5,15.438443,1041.812626
6,6,14.918056,2520.528413


# 8.9 Simple Cross-Validation

No need to wait for new data, can cross-validate with data you already have. Simple cross-validation:
- Break up data into 2 random sets, non overlapped
- Training set: to train the model
- Development set: to evaluate the model after training
    - AKA validation test, cross-validation set

## Perform the Split

In [242]:
from sklearn.utils import shuffle

(training, dev) = np.split(shuffle(vehicle_data_35), [25])
display(training.head())
display(dev.head())

,mpg,cylinders,displacement,hp,weight,acceleration,model year,origin,car name
358,31.6,4,120.0,74.0,2635,18.3,81,3,mazda 626
34,16.0,6,225.0,105.0,3439,15.5,71,1,plymouth satellite custom
394,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup
210,19.0,6,156.0,108.0,2930,15.5,76,3,toyota mark ii
393,27.0,4,140.0,86.0,2790,15.6,82,1,ford mustang gl


,mpg,cylinders,displacement,hp,weight,acceleration,model year,origin,car name
205,28.0,4,97.0,75.0,2155,16.4,76,3,toyota corolla
289,16.9,8,350.0,155.0,4360,14.9,79,1,buick estate wagon (sw)
79,26.0,4,96.0,69.0,2189,18.0,72,2,renault 12 (sw)
15,22.0,6,198.0,95.0,2833,15.5,70,1,plymouth duster
167,29.0,4,97.0,75.0,2171,16.0,75,3,toyota corolla


## Train on the Training Set

In [246]:
(training, dev) = np.split(shuffle(vehicle_data_35), [25])
df_mse_fit = pd.DataFrame(
    {"Degree": ks, "MSE": [get_mse_for_degree_k(k, df_fit=training) for k in ks]}
)
df_mse_eval = pd.DataFrame(
    {
        "Validation MSE": [
            get_mse_for_degree_k(k, df_fit=training, df_evaluate=dev) for k in ks
        ],
    }
)

display(df_mse_fit.join(df_mse_eval))

,Degree,MSE,Validation MSE
0,0,51.070864,66.611784
1,1,20.062888,28.882545
2,2,18.856241,25.474424
3,3,18.984886,31.201120
4,4,18.412370,29.645850
5,5,17.324055,27.004130
6,6,16.857560,26.078855


## Takeaway: Minimize the validation error, not the training error

![alt text](./data/cross_validation.png "Cross Validation")

## Terminology: hyperparameter

hyperparameter: parameter that controls the learning process itself  
Example: degree of polynomial

So we used the training set to select parameteres, and we used the validation set to choose hyperparameter

# 8.10 Test Sets

Performing the process of training and development is itself biased, because you choose the parameters and hyperparameters that optimize performance - you chose the best performer so it is biased in the sense of not enirely honest.

Enter the **test set**! A 3rd dataset that is different than the training and development sets that is used to state the model's final performance
- Unbiased in the sense that you made no choices based on this data set


## Perform a 3-way split

In [251]:
(training, dev, test) = np.split(shuffle(vehicle_data_35), [20, 30])
[training.shape, dev.shape, test.shape]

[(20, 9), (10, 9), (5, 9)]

## Necessity of a Test Set

You don't need it if you just want to choose the best performing model.  
- You only need it if you also want to provide an unbiased estimate of its performance
- If you added a performance curve for the test set vs. complexity, it would follow similar trend to that of the development set
    - However, never do this! Only want to use the test after the fact

# 8.2 Mini Lesson

<img src = 'data/training_validation_test_rules.png'/><img